In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import json
import joblib


In [2]:
def set_project_directory():
    current_dir = os.getcwd()
    
    if os.path.basename(current_dir) == 'scripts':
        os.chdir('..')
    
    print(f"Working directory set to: {os.getcwd()}")

    
set_project_directory()

Working directory set to: c:\Users\Dana\Documents\Kuliah\Bangkit\Capstone-C242-PS384_Project01


In [3]:
folder_path = 'models/'

try:
    print("Daftar file di dalam folder:", os.listdir(folder_path))
    required_files = [
        'disease-prediction-model.joblib',
        'disease-prediction-model.hdf5',
        'disease-prediction-model.json'
    ]
    
    missing_files = [f for f in required_files if not os.path.isfile(os.path.join(folder_path, f))]
    
    if missing_files:
        print(f"File yang hilang: {', '.join(missing_files)}")
    else:
        print("Semua file dataset ditemukan.")

except FileNotFoundError:
    print(f"Folder tidak ditemukan: {folder_path}")

Daftar file di dalam folder: ['disease-prediction-model.hdf5', 'disease-prediction-model.joblib', 'disease-prediction-model.json', 'scaler.joblib']
Semua file dataset ditemukan.


In [6]:
def load_model_and_params(model_path='models/disease-prediction-model.joblib', 
                         scaler_path='models/scaler.joblib'):
    """Load the trained model and its parameters"""
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        raise FileNotFoundError(f"Model or scaler file not found")
    
    # Load model and scaler separately
    models = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    
    return models, scaler

def calculate_bmi(weight, height):
    """Calculate BMI from weight (kg) and height (cm)"""
    if height > 0:
        return weight / ((height / 100) ** 2)
    else:
        return np.nan

def get_bmi_category(bmi):
    """Get BMI category"""
    if bmi < 18.5:
        return "Underweight"
    elif bmi < 25:
        return "Normal"
    elif bmi < 30:
        return "Overweight"
    else:
        return "Obese"

def get_risk_level(probability):
    """Get risk level based on probability"""
    if probability < 20:
        return "Risiko Rendah"
    elif probability < 40:
        return "Risiko Sedang"
    elif probability < 60:
        return "Risiko Tinggi"
    else:
        return "Risiko Sangat Tinggi"
    
    

In [11]:
def predict_health_risks(user_input, models, scaler):
    """Predict health risks for user input"""
    # Calculate BMI
    bmi = calculate_bmi(user_input['weight'], user_input['height'])
    
    # Prepare features with same columns as training data
    features = pd.DataFrame([[
        user_input['height'],
        user_input['weight'], 
        user_input['gender'],
        user_input['age'],
        user_input['blood_pressure'],  # bp
        user_input['cholesterol'],     # bc
        user_input['blood_glucose'],   # bg
        bmi,
        0,    # sodium (default)
        0,    # fat (default)
        0,    # protein (default)
        0     # carbs (default)
    ]])
    
    # Set column names to match training data
    features.columns = [
        'height', 'weight', 'gender', 'age', 'bp', 'bc', 'bg', 'bmi',
        'sodium', 'fat', 'protein', 'carbs'
    ]
    
    # Scale features
    features_scaled = scaler.transform(features)
    
    # Make predictions for each disease
    predictions = {}
    for disease, model in models.items():
        try:
            proba = model.predict_proba(features_scaled)[0]
            prob = proba[1] if len(proba) > 1 else model.predict(features_scaled)[0]
        except:
            prob = model.predict(features_scaled)[0]
        predictions[disease] = float(prob) * 100
    
    return bmi, predictions

In [12]:
def main():
    try:
        # Load models and scaler
        models, scaler = load_model_and_params()
        
        # Get user input
        print("\n=== Health Risk Prediction System ===")
        user_input = {
            'height':170               #float(input("Masukkan tinggi badan (cm): "))
            ,'weight':60              #float(input("Masukkan berat badan (kg): "))
            ,'gender':1              #1 if input("Masukkan jenis kelamin (L/P): ").upper() == 'L' else 0
            ,'age':25                 #float(input("Masukkan usia: "))
            ,'blood_pressure':120      #float(input("Masukkan tekanan darah (mmHg): "))
            ,'cholesterol':180         #float(input("Masukkan kolesterol (mg/dL): "))
            ,'blood_glucose':90       #float(input("Masukkan gula darah (mg/dL): "))
        }
        
        # Make predictions
        bmi, predictions = predict_health_risks(user_input, models, scaler)
        
        # Display results
        print("\n=== Hasil Prediksi ===")
        print(f"\nBMI: {bmi:.1f}")
        print(f"Kategori BMI: {get_bmi_category(bmi)}")
        
        print("\nRisiko Penyakit:")
        for disease, risk in predictions.items():
            disease_name = disease.replace('_', ' ').title()
            risk_level = get_risk_level(risk)
            print(f"{disease_name}: {risk:.1f}% ({risk_level})")
            
    except Exception as e:
        print(f"\nError: {str(e)}")
        print("Pastikan model sudah dilatih dan disimpan di folder 'models/'")

if __name__ == "__main__":
    main()

c:\Users\Dana\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Dana\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



=== Health Risk Prediction System ===

Error: X has 12 features, but StandardScaler is expecting 9 features as input.
Pastikan model sudah dilatih dan disimpan di folder 'models/'


c:\Users\Dana\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Dana\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
